In [1]:
%%time
import xarray as xr
import numpy as np
import skimage.measure
import pandas as pd
from scipy.spatial import distance
import skimage.measure
from skimage.morphology import remove_small_objects,closing,binary_closing
from scipy import ndimage
from datetime import datetime,timedelta
import os
from dask.distributed import Client
import glob
from shapely.geometry import MultiPoint
import geopandas as gpd
import cartopy.crs as ccrs

client = Client()

# Create a path to the code file
codeDir = os.path.dirname(os.path.abspath(os.getcwd()))
parentDir = os.path.dirname(codeDir)

mrms_folder = os.path.join(parentDir,"MRMS","2min_rate_cat_month_CO","eval")
storm_folder = os.path.join(parentDir,"CO_storm_output")

os.chdir(mrms_folder)
filenames_rate = glob.glob('*.grib2')
os.chdir(storm_folder)
filenames_storm = glob.glob('*.nc')


month = xr.open_dataset(mrms_folder+'\\'+filenames_rate[1], engine = "cfgrib",chunks={'time': '500MB'})
month = month.where(month.longitude<=256,drop=True)

storm = xr.open_dataset(storm_folder+'\\'+filenames_storm[1],chunks={'time': '500MB'})
start=month.time[0].values
end=start+np.timedelta64(1,'D')
# create test slice of time
month = month.sel(time=slice(start,end))
storm = storm.sel(time=slice(start,end))

# get storm ids
storms = storm.storm_id.values
storms = storms[~np.isnan(storms)]
storms = storms[storms>0]
storms = np.unique(storms)

# split into sections
sample = np.array_split(storms,20)

velocity = []
duration = []
storm_accum = []
storm_id=[]

for i in range(1):
    sample_storms = storm.where(storm.storm_id.isin(sample[i]),drop=True)
    sample_month = month.sel(time=sample_storms.time,latitude = sample_storms.latitude, 
                             longitude = sample_storms.longitude,method='nearest')
    sample_month = sample_month.assign({"storm_id": sample_storms.storm_id})

    sample_month = sample_month.to_dataframe()
    sample_month['unknown'] = sample_month.unknown*(2/60)

    for s in sample[0]:
        accum = sample_month.loc[sample_month.storm_id==s].drop(['step','heightAboveSea','valid_time'],
                                                        axis=1).reset_index().groupby(['latitude','longitude']).sum().unknown
        accum['storm_id']=s
        storm_accum.append(accum)

        agg = sample_month.loc[sample_month.storm_id==s].drop(['step','heightAboveSea','valid_time'],
                                                            axis=1).reset_index().groupby(['time']).agg(list)
        centroid = []
        for j in range(len(agg)):
            latitude = agg.latitude[j]
            longitude = agg.longitude[j]
            gage_coord = np.stack((longitude,latitude),axis=1)
            points = MultiPoint(gage_coord)
            centroid.append(points.centroid) 

        # add nan to start of list to comare distance from last point
        if len(centroid)<=1:
            distance=np.nan
            duration.append([s,2])
        else:
            fill = [np.nan]
            centroid = fill+centroid
            centroid_last = centroid[0:-1]
            centroid = centroid[1:len(centroid)]
            # reproject to get accurate distance
            centroid = gpd.GeoSeries(centroid, crs='EPSG:4326')
            centroid_last = gpd.GeoSeries(centroid_last, crs='EPSG:4326')
            crs = ccrs.LambertConformal(central_latitude=38.5, central_longitude=-105)
            centroid = centroid.to_crs(crs)
            centroid_last = centroid.shift()
            distance = centroid.distance(centroid_last)

            storm_id.append(s)
            velocity.append((distance/(2*60)).mean())

            dur_timestamp = agg.index[-1]-agg.index[0]
            dur_sec = dur_timestamp.seconds
            dur_min = (dur_sec % 3600) // 60
            duration.append(dur_min)


Ignoring index file 'Z:\\PhoebeRadar\\MRMS\\2min_rate_cat_month_CO\\eval\\aug_2022_rate_CO.grib2.923a8.idx' incompatible with GRIB file
C:\Users\whitep\Miniconda3\envs\radar\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: total: 1min 54s
Wall time: 4min 23s


In [1]:
%%time
import xarray as xr
import numpy as np
import skimage.measure
import pandas as pd
from scipy.spatial import distance
import skimage.measure
from skimage.morphology import remove_small_objects,closing,binary_closing
from scipy import ndimage
from datetime import datetime,timedelta
import os
from dask.distributed import Client
import glob
from shapely.geometry import MultiPoint
import geopandas as gpd
import cartopy.crs as ccrs

client = Client()

# Create a path to the code file
codeDir = os.path.dirname(os.path.abspath(os.getcwd()))
parentDir = os.path.dirname(codeDir)

mrms_folder = os.path.join(parentDir,"MRMS","2min_rate_cat_month_CO","eval")
storm_folder = os.path.join(parentDir,"CO_storm_output")

os.chdir(mrms_folder)
filenames_rate = glob.glob('*.grib2')
os.chdir(storm_folder)
filenames_storm = glob.glob('*.nc')


month = xr.open_dataset(mrms_folder+'\\'+filenames_rate[1], engine = "cfgrib",chunks={'time': '10MB'})
month = month.where(month.longitude<=256,drop=True)

storm = xr.open_dataset(storm_folder+'\\'+filenames_storm[1],chunks={'time': '10MB'})
start=month.time[0].values
end=start+np.timedelta64(1,'D')
# create test slice of time
month = month.sel(time=slice(start,end))
storm = storm.sel(time=slice(start,end))

# get storm ids
storms = storm.storm_id.values
storms = storms[~np.isnan(storms)]
storms = storms[storms>0]
storms = np.unique(storms)

# split into sections
sample = np.array_split(storms,20)

velocity = []
duration = []
storm_accum = []
storm_id=[]



Ignoring index file 'Z:\\PhoebeRadar\\MRMS\\2min_rate_cat_month_CO\\eval\\aug_2022_rate_CO.grib2.923a8.idx' incompatible with GRIB file
C:\Users\whitep\Miniconda3\envs\radar\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: total: 1min 13s
Wall time: 1min 22s


In [ ]:
%%time
for i in range(1):
    sample_storms = storm.where(storm.storm_id.isin(sample[i]),drop=True)
    sample_month = month.sel(time=sample_storms.time,latitude = sample_storms.latitude, 
                             longitude = sample_storms.longitude,method='nearest')
    sample_month = sample_month.assign({"storm_id": sample_storms.storm_id})

    sample_month = sample_month.to_dataframe()
    sample_month['unknown'] = sample_month.unknown*(2/60)

    for s in sample[0]:
        accum = sample_month.loc[sample_month.storm_id==s].drop(['step','heightAboveSea','valid_time'],
                                                        axis=1).reset_index().groupby(['latitude','longitude']).sum().unknown
        accum['storm_id']=s
        storm_accum.append(accum)

        agg = sample_month.loc[sample_month.storm_id==s].drop(['step','heightAboveSea'],
                                                            axis=1).reset_index().groupby(['time']).agg(list)
        agg = agg.resample('30min').agg(sum)
        
        centroid = []
        for j in range(len(agg)):
            latitude = agg.latitude[j]
            longitude = agg.longitude[j]
            gage_coord = np.stack((longitude,latitude),axis=1)
            points = MultiPoint(gage_coord)
            centroid.append(points.centroid) 

        # add nan to start of list to comare distance from last point
        if len(centroid)<=1:
            distance=np.nan
            #duration.append([s,2])
        else:
            fill = [np.nan]
            centroid = fill+centroid
            centroid_last = centroid[0:-1]
            centroid = centroid[1:len(centroid)]
            # reproject to get accurate distance
            centroid = gpd.GeoSeries(centroid, crs='EPSG:4326')
            centroid_last = gpd.GeoSeries(centroid_last, crs='EPSG:4326')
            crs = ccrs.LambertConformal(central_latitude=38.5, central_longitude=-105)
            centroid = centroid.to_crs(crs)
            centroid_last = centroid.shift()
            distance = centroid.distance(centroid_last)

            storm_id.append(s)
            velocity.append((distance/(30*60)).mean())

   


In [85]:
agg.latitude.values

array([list([40.57499999999929, 40.58499999999929, 40.57499999999929, 40.58499999999929])],
      dtype=object)

In [87]:
agg

,latitude,longitude
time,,
2022-08-01,"[40.57499999999929, 40.58499999999929, 40.5749...","[253.4749989999977, 253.4749989999977, 253.474..."


In [86]:
np.mean(velocity)

1.3599815091146419

In [56]:
np.mean(velocity)

0.9833727393536313

In [76]:
for i in range(1):
    sample_storms = storm.where(storm.storm_id.isin(sample[i]),drop=True)
    sample_month = month.sel(time=sample_storms.time,latitude = sample_storms.latitude, 
                             longitude = sample_storms.longitude,method='nearest')
    sample_month = sample_month.assign({"storm_id": sample_storms.storm_id})

    sample_month = sample_month.to_dataframe()
    sample_month['unknown'] = sample_month.unknown*(2/60)

    for s in sample[0]:
        accum = sample_month.loc[sample_month.storm_id==s].drop(['step','heightAboveSea','valid_time'],
                                                        axis=1).reset_index().groupby(['latitude','longitude']).sum().unknown
        accum['storm_id']=s
        storm_accum.append(accum)

        agg = sample_month.loc[sample_month.storm_id==s].drop(['step','heightAboveSea','valid_time'],
                                                            axis=1).reset_index().groupby(['time']).agg(list)
        centroid = []
        for j in range(len(agg)):
            latitude = agg.latitude[j]
            longitude = agg.longitude[j]
            gage_coord = np.stack((longitude,latitude),axis=1)
            points = MultiPoint(gage_coord)
            centroid.append(points.centroid) 

        # add nan to start of list to comare distance from last point
        if len(centroid)<=1:
            distance=np.nan
            duration.append([s,2])
        else:
            fill = [np.nan]
            centroid = fill+centroid
            centroid_last = centroid[0:-1]
            centroid = centroid[1:len(centroid)]
            # reproject to get accurate distance
            centroid = gpd.GeoSeries(centroid, crs='EPSG:4326')
            centroid_last = gpd.GeoSeries(centroid_last, crs='EPSG:4326')
            crs = ccrs.LambertConformal(central_latitude=38.5, central_longitude=-105)
            centroid = centroid.to_crs(crs)
            centroid_last = centroid.shift()
            distance = centroid.distance(centroid_last)

            storm_id.append(s)
            velocity.append((distance/(2*60)).mean())

            dur_timestamp = agg.index[-1]-agg.index[0]
            dur_sec = dur_timestamp.seconds
            dur_min = (dur_sec % 3600) // 60
            duration.append(dur_min)
